In [1]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd

In [2]:
def process_csv_file(file):
    csv_data = pd.read_csv(file)
    csv_data = csv_data.drop("class", axis = 1)
    return (csv_data.to_numpy())

In [3]:
def set_padding(arr, max_length):
    arr = np.append(arr, np.zeros((max_length-arr.shape[0],300)), axis=0)
    return np.expand_dims(arr, 0)

In [4]:
def prepare_data(folder_path, labels):
    total = 0
    max_length = 0
    # find max length
    for folder in os.listdir(folder_path):
        if folder != ".DS_Store":
            total += len(os.listdir(os.path.join(folder_path, folder)))
            for file in os.listdir(os.path.join(folder_path, folder)):
                data = process_csv_file(os.path.join(folder_path, folder, file))
                if data.shape[0] > max_length:
                    max_length = data.shape[0]
    # Make the arrays
    X = np.empty((0, max_length, 300))
    Y = np.empty((0,), int)
    for folder in os.listdir(folder_path):
        print(folder)
        if folder != ".DS_Store":
            for file in os.listdir(os.path.join(folder_path, folder)):
                data = process_csv_file(os.path.join(folder_path, folder, file))
                data = set_padding(data, max_length)
                X = np.vstack((X, data))
                Y = np.append(Y, labels[folder])
    print(total, max_length)
    return X, Y


In [20]:
folder_path = "/Users/aly/Documents/Programming/Apps/Machine Learning/ASL Converter/training_models/mediapipe/reformatting-the-data/data_four_labels/"
labels= {"coffee": 0, 'dog': 1, 'milk': 2, 'door': 3}
X, Y = prepare_data(folder_path, labels)

.DS_Store
dog
milk
coffee
door
143 211


In [6]:
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
], padding="post"
)
# print(X[0][1])
print(((padded_inputs)))

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]


In [21]:
# Y = np.expand_dims(Y, axis=0)
# Y= Y.T
Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [22]:
# split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [23]:
X_train.shape, y_train.shape

((114, 211, 300), (114,))

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])




Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-03-20 13:48:08.831618: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-20 13:48:08.831726: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


NameError: name 'X_train' is not defined

In [41]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(), metrics=["accuracy"])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_33 (LSTM)              (None, 211, 64)           93440     
                                                                 
 lstm_34 (LSTM)              (None, 32)                12416     
                                                                 
 dense_7 (Dense)             (None, 4)                 132       
                                                                 
Total params: 105,988
Trainable params: 105,988
Non-trainable params: 0
_________________________________________________________________


In [42]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/10


2023-03-18 02:36:51.372559: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:51.660659: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:51.934093: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:52.419899: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:53.015099: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - ETA: 0s - loss: 1.3853 - accuracy: 0.2353

2023-03-18 02:36:55.555320: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:55.667921: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:55.926320: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 6s 863ms/step - loss: 1.3853 - accuracy: 0.2353 - val_loss: 1.3813 - val_accuracy: 0.3333
Epoch 2/10
4/4 [==============================] - 0s 108ms/step - loss: 1.3759 - accuracy: 0.3333 - val_loss: 1.3774 - val_accuracy: 0.3333
Epoch 3/10
4/4 [==============================] - 0s 99ms/step - loss: 1.3661 - accuracy: 0.3431 - val_loss: 1.3739 - val_accuracy: 0.3333
Epoch 4/10
4/4 [==============================] - 0s 101ms/step - loss: 1.3470 - accuracy: 0.3431 - val_loss: 1.3667 - val_accuracy: 0.3333
Epoch 5/10
4/4 [==============================] - 0s 101ms/step - loss: 1.3325 - accuracy: 0.3431 - val_loss: 1.3640 - val_accuracy: 0.3333
Epoch 6/10
4/4 [==============================] - 0s 101ms/step - loss: 1.3215 - accuracy: 0.3431 - val_loss: 1.4380 - val_accuracy: 0.3333
Epoch 7/10
4/4 [==============================] - 0s 102ms/step - loss: 1.2628 - accuracy: 0.4118 - val_loss: 2.1948 - val_accuracy: 0.1667
Epoch 8/10
4/4 [================

In [38]:
x = np.array([[4], [2]])

In [41]:
x.T.shape

(1, 2)

In [42]:
x.T

array([[4, 2]])